### Ensambles

En este notebook se probarán 3 algoritmos diferentes basados en árboles:

* Árbol de decisión (Decision Tree)
* Bosques aleatorios (Random Forest)
* Árboles potenciados por gradiente (Gradient Boosting)

---

#### Conjunto de Datos

Usaremos un conjunto de datos del sector educación para clasificar a las universidades como privadas o públicas en función de estas características:

* School: Nombre de la institución
* Private: Un factor con niveles No y Sí que indican universidad pública o privada
* Apps: Número de solicitudes recibidas
* Accept: Número de solicitudes aceptadas
* Enroll: Número de nuevos estudiantes matriculados
* Top10perc: Porcentaje de nuevos estudiantes del top 10% de las clases de preparatoria
* Top25perc: Porcentaje de nuevos estudiantes del top 25% de las clases de preparatoria
* F_Undergrad: Número de estudiantes universitarios de tiempo completo
* P_Undergrad: Número de estudiantes universitarios a tiempo parcial
* Outstate: Matrícula fuera del estado
* Room_Board: Costos de alojamiento y comida
* Books: Costos estimados de libros
* Personal: Gastos personales estimados
* PhD: Porcentaje de profesores con doctorado
* Terminal: Porcentaje de profesores con título terminal (Terminal degree)
* S.F.Ratio: Proporción estudiantes/profesores
* Perc_alumni: Porcentaje de ex alumnos que donan
* Expend: Gastos por alumno (Incluye salarios y beneficios para maestros, servicios y gastos relacionados con actividades extracurriculares).
* Grad_Rate: Tasa de graduación

In [0]:
generation = "mod4gen13"

In [0]:
data = spark.read.csv(f"/mnt/{generation}/input/college.csv", inferSchema=True, header=True)

In [0]:
data.printSchema()

In [0]:
data.count(), len(data.columns)

In [0]:
data.display()

In [0]:
data.show(n=5)

In [0]:
data.show(n=5, truncate=False)

### EDA

In [0]:
data.groupBy("Private").count().display()

In [0]:
data.groupBy("Private").count().display()

### Preprocesamiento

In [0]:
from pyspark.ml.feature import VectorAssembler

In [0]:
cols = data.columns[2:]
cols

In [0]:
assembler = VectorAssembler(inputCols=cols, outputCol="features")

In [0]:
type(assembler)

In [0]:
output = assembler.transform(data)

In [0]:
output.display()

In [0]:
from pyspark.ml.feature import StringIndexer

In [0]:
indexer = StringIndexer(inputCol="Private", outputCol="PrivateIndex")

In [0]:
type(indexer)

In [0]:
output_fixed = indexer.fit(output).transform(output)

In [0]:
output_fixed.display()

In [0]:
final_data = output_fixed.select("features", "PrivateIndex")

In [0]:
final_data.display()

In [0]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

### Modelación

In [0]:
from pyspark.ml.classification import DecisionTreeClassifier, GBTClassifier, RandomForestClassifier

In [0]:
help(RandomForestClassifier)

In [0]:
# Se utilizan los hiperparámetros predeterminados para que la comparación sea "justa"

dtc = DecisionTreeClassifier(labelCol="PrivateIndex", featuresCol="features")
rfc = RandomForestClassifier(labelCol="PrivateIndex", featuresCol="features")
gbt = GBTClassifier(labelCol="PrivateIndex", featuresCol="features")

In [0]:
dtc_model = dtc.fit(train_data)
rfc_model = rfc.fit(train_data)
gbt_model = gbt.fit(train_data)

### Importancia de variables

In [0]:
dtc_model.featureImportances

In [0]:
len(cols), len(rfc_model.featureImportances)

In [0]:
list(zip(cols, gbt_model.featureImportances))

### Comparativa de modelos

In [0]:
dtc_predictions = dtc_model.transform(test_data)
rfc_predictions = rfc_model.transform(test_data)
gbt_predictions = gbt_model.transform(test_data)

In [0]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

[MulticlassClassificationEvaluator](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.evaluation.MulticlassClassificationEvaluator.html)

In [0]:
# Se evalúa el real contra el predicho calculando el error en la porción de prueba
f1_evaluator = MulticlassClassificationEvaluator(labelCol="PrivateIndex", predictionCol="prediction", metricName="f1")
# acc_evaluator = MulticlassClassificationEvaluator(labelCol="PrivateIndex", predictionCol="prediction", metricName="accuracy")

In [0]:
import numpy as np

In [0]:
dtc_f1 = f1_evaluator.evaluate(dtc_predictions)
rfc_f1 = f1_evaluator.evaluate(rfc_predictions)
gbt_f1 = f1_evaluator.evaluate(gbt_predictions)

In [0]:
print(f"F1-score: Decisión Tree: {np.round(dtc_f1, 2)}")
print(f"F1-score: Random Forest: {np.round(rfc_f1, 2)}")
print(f"F1-score: Grad Boosting: {np.round(gbt_f1, 2)}")